# Refactor the Taxi Fare Prediction Problem with a Pipeline

We will refactor the model you built for the Taxi Fare Prediction Problem using:
- Custom encoders for the distance and time features
- OneHot Encoder in order to encode the hour and day of week features
- SimpleImputer to fill missing values
- A simple linear regression
- A pipeline to put all together

Then: 
- train this pipeline
- apply the pipeline on test data
- generate predictions and submit these new predictions to Kaggle

## First pipeline

In [1]:
import pandas as pd

data = pd.read_csv("data/train.csv", nrows=1000)
data.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1


In [2]:
# prepare X and y
X = data.drop(columns=["fare_amount"])
Y = data["fare_amount"]
X.head()

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1


In [3]:
Y.head()

0     4.5
1    16.9
2     5.7
3     7.7
4     5.3
Name: fare_amount, dtype: float64

In [4]:
# Hold out ( train and test dplit )
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)

In [5]:
X_train.head()

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
364,2013-12-18 09:15:00.000000217,2013-12-18 09:15:00 UTC,-73.979410,40.766107,-74.003015,40.750522,1
699,2013-07-11 21:10:00.00000060,2013-07-11 21:10:00 UTC,-73.978007,40.745810,-73.984250,40.733410,2
783,2010-12-15 22:18:27.0000003,2010-12-15 22:18:27 UTC,-73.985287,40.773551,-73.964319,40.809258,1
315,2015-03-19 12:38:17.0000001,2015-03-19 12:38:17 UTC,-73.873055,40.774086,-73.982727,40.771496,1
270,2012-08-25 01:53:42.0000005,2012-08-25 01:53:42 UTC,-73.995892,40.746454,-73.995892,40.746454,1


### Custom transformers

With the Taxi Fare Prediction Challenge data, using `BaseEstimator` and `TransformerMixin`, implement:

- a transformer that computes the haversine distance between the pickup and dropoff locations
- a custom encoder that extracts the time features from `pickup_datetime`

In [6]:
import numpy as np

def haversine_vectorized(df, 
                         start_lat="pickup_latitude",
                         start_lon="pickup_longitude",
                         end_lat="dropoff_latitude",
                         end_lon="dropoff_longitude"):
    """ 
        Calculates the great circle distance between two points 
        on the earth (specified in decimal degrees).
        Vectorized version of the haversine distance for pandas df.
        Computes the distance in kms.
    """

    lat_1_rad, lon_1_rad = np.radians(df[start_lat].astype(float)), np.radians(df[start_lon].astype(float))
    lat_2_rad, lon_2_rad = np.radians(df[end_lat].astype(float)), np.radians(df[end_lon].astype(float))
    dlon = lon_2_rad - lon_1_rad
    dlat = lat_2_rad - lat_1_rad

    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat_1_rad) * np.cos(lat_2_rad) * np.sin(dlon / 2.0) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    return 6371 * c

In [7]:
# create a DistanceTransformer
from sklearn.base import BaseEstimator, TransformerMixin

class DistanceTransformer(TransformerMixin, BaseEstimator):
    """
        Computes the haversine distance between two GPS points.
        Returns a copy of the DataFrame X with only one column: 'distance'.
        CORRECTION: returns the whole dataframe (impossible to use both pipelines
        at once otherwise)
    """

    def __init__(self,
                 start_lat="pickup_latitude",
                 start_lon="pickup_longitude",
                 end_lat="dropoff_latitude",
                 end_lon="dropoff_longitude"):
        self.start_lat = start_lat
        self.start_lon = start_lon
        self.end_lat = end_lat
        self.end_lon = end_lon
        
    def fit(self, X, y=None): return self

    def transform(self, X, y=None):
    
        X_ = X.copy()
        distance = haversine_vectorized(X)
        X_["distance"] = distance
        return X_[["distance"]]

In [8]:
# test the DistanceTransformer

dist_trans = DistanceTransformer()
distance = dist_trans.fit_transform(X_train, y_train)
distance.head()

,distance
364,2.637429
699,1.475733
783,4.345143
315,9.239850
270,0.000000


In [9]:
# create a TimeFeaturesEncoder
class TimeFeaturesEncoder(BaseEstimator, TransformerMixin):
    """
        Extracts the day of week (dow), the hour, the month and the year from a time column.
        Returns a copy of the DataFrame X with only four columns: 'dow', 'hour', 'month', 'year'.
    """

    def __init__(self, time_column_name="pickup_datetime"):
        self.time_column_name = time_column_name
        self.utc_time_column = time_column_name
        self.NY_time_column = time_column_name + "_NY_time"
    
    def __extract_time_features(self, df):
        
        df[self.utc_time_column] = pd.to_datetime(df[self.utc_time_column], 
                                               infer_datetime_format=True
                                          )
        index = df[self.utc_time_column]
        df[self.NY_time_column] = df[self.utc_time_column].dt.tz_convert("US/Eastern")
        df["hour"] = df[self.NY_time_column].dt.hour
        df["dow"] = df[self.NY_time_column].dt.dayofweek
        df["month"] = df[self.NY_time_column].dt.month
        df["year"] = df[self.NY_time_column].dt.year
        df.set_index(self.utc_time_column, inplace=True)
        return df

    def fit(self, X, y=None): return self
        

    def transform(self, X, y=None):
        X_ = X.copy()
        X_ = self.__extract_time_features(X_)
        return X_[["hour", "dow", "month", "year"]]

In [10]:
# test the TimeFeaturesEncoder

time_enc = TimeFeaturesEncoder('pickup_datetime')
time_features = time_enc.fit_transform(X_train, y_train)
time_features.head()

,hour,dow,month,year
pickup_datetime,,,,
2013-12-18 09:15:00+00:00,4,2,12,2013
2013-07-11 21:10:00+00:00,17,3,7,2013
2010-12-15 22:18:27+00:00,17,2,12,2010
2015-03-19 12:38:17+00:00,8,3,3,2015
2012-08-25 01:53:42+00:00,21,4,8,2012


###  Prepocessing pipeline

In [11]:
# visualizing pipelines in HTML
from sklearn import set_config; set_config(display='diagram')

#### Distance pipeline

Create a pipeline for distances:
- convert the pickup and dropoff coordinates into distances with the DistanceTransformer
- standardize these distances

In [12]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

dist_pipe = Pipeline([('dist_trans', DistanceTransformer()),
                     ('stdscaler', StandardScaler())])

dist_pipe


Pipeline(steps=[('dist_trans', DistanceTransformer()),
                ('stdscaler', StandardScaler())])

#### Time features pipeline

Create a pipeline for time features
- extract time features from pickup datetime with the TimeFeaturesEncoder
- encode these categorical time features with the OneHotEncoder

In [13]:
from sklearn.preprocessing import OneHotEncoder

time_pipe = Pipeline([('time_enc', TimeFeaturesEncoder('pickup_datetime')),
                     ('ohe', OneHotEncoder(handle_unknown='ignore'))])


time_pipe


Pipeline(steps=[('time_enc', TimeFeaturesEncoder()),
                ('ohe', OneHotEncoder(handle_unknown='ignore'))])

#### Preprocessing pipeline

Wrap up the distance pipeline and the time pipeline into a preprocessing pipeline.

In [14]:
from sklearn.compose import ColumnTransformer
preproc_pipe = ColumnTransformer(transformers=[
    ("distance", dist_pipe, ["pickup_latitude", 
                                     "pickup_longitude", 
                                     "dropoff_latitude",
                                    "dropoff_longitude"]),
    ("time", time_pipe, ["pickup_datetime"])
])

preproc_pipe


ColumnTransformer(transformers=[('distance',
                                 Pipeline(steps=[('dist_trans',
                                                  DistanceTransformer()),
                                                 ('stdscaler',
                                                  StandardScaler())]),
                                 ['pickup_latitude', 'pickup_longitude',
                                  'dropoff_latitude', 'dropoff_longitude']),
                                ('time',
                                 Pipeline(steps=[('time_enc',
                                                  TimeFeaturesEncoder()),
                                                 ('ohe',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['pickup_datetime'])])

### Model pipeline

Create a pipeline containing the preprocessing and the regression model of your choice.

In [15]:
from sklearn.linear_model import LinearRegression

pipe = Pipeline([
    ('preproc', preproc_pipe),
    ('linear_model', LinearRegression())
])

pipe


Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('distance',
                                                  Pipeline(steps=[('dist_trans',
                                                                   DistanceTransformer()),
                                                                  ('stdscaler',
                                                                   StandardScaler())]),
                                                  ['pickup_latitude',
                                                   'pickup_longitude',
                                                   'dropoff_latitude',
                                                   'dropoff_longitude']),
                                                 ('time',
                                                  Pipeline(steps=[('time_enc',
                                                                   TimeFeaturesEncoder()),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['pickup_datetime'])])),
                ('linear_model', LinearRegression())])

### Training and performance

Train the pipelined model and compute the prediction on the test set:

In [16]:
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)


Use the RMSE to evaluate the performance of the model:

In [17]:
def compute_rmse(y_pred, y_true):
    mse = np.mean((y_pred - y_true)**2)
    return np.sqrt(mse)

In [18]:
# call compute_rmse
compute_rmse(y_pred, y_test)

9.976539684120706

## Complete the workflow with a pipeline

Here we will implement the whole workflow for our Taxifare kaggle challenge.

For that we will refactor the code in functions for more clarity.

Implement the following functions:
- `get_data()` to fetch the data 
- `clean_data()` to clean the data
- `get_pipeline()` to get the pipeline defined earlier
- `train()` to train our model
- `evaluate()` to evaluate our model on test data

In [19]:
# implement get_data() function
def get_data(nrows=10000):
    df = pd.read_csv("data/train.csv", nrows=nrows)
    return df

In [20]:
# implement clean_data() function
def clean_data(df, test=False):
    '''returns a DataFrame without outliers and missing values'''
    df.dropna(inplace=True)
    df = df[df["passenger_count"].between(0, 12)]
    df = df[df["pickup_latitude"].between(left = 40, right = 42 )]
    df = df[df["pickup_longitude"].between(left = -74.3, right = -72.9 )]
    df = df[df["dropoff_latitude"].between(left = 40, right = 42 )]
    df = df[df["dropoff_longitude"].between(left = -74, right = -72.9 )]
    df = df[df["fare_amount"] > 0]
    return df

In [21]:
# implement set_pipeline() function
def set_pipeline():
    '''returns a pipelined model'''
    dist_pipe = Pipeline([('dist_trans', DistanceTransformer()),
                     ('stdscaler', StandardScaler())])
    time_pipe = Pipeline([('time_enc', TimeFeaturesEncoder('pickup_datetime')),
                     ('ohe', OneHotEncoder(handle_unknown='ignore'))])
    preproc_pipe = ColumnTransformer(transformers=[
    ("distance", dist_pipe, ["pickup_latitude", 
                                     "pickup_longitude", 
                                     "dropoff_latitude",
                                    "dropoff_longitude"]),
    ("time", time_pipe, ["pickup_datetime"])
    ])
    pipe = Pipeline([
    ('preproc', preproc_pipe),
    ('linear_model', LinearRegression())
    ])
    return pipe

In [22]:
# implement train() function
def train(X_train, y_train, pipeline):
    '''returns a trained pipelined model'''
    pipe.fit(X_train, y_train)
    return pipeline

In [23]:
# implement evaluate() function
def evaluate(X_test, y_test, pipeline):
    '''returns the value of the RMSE'''
    y_pred = pipe.predict(X_test)
    rmse = compute_rmse(y_pred, y_test)
    return rmse

### Test the complete worflow

Use the above functions to test the complete workflow.

In [24]:
# store the data in a DataFrame
df = get_data()
df = clean_data(df)

# set X and y
y = df["fare_amount"]
X = df.drop("fare_amount", axis=1)

# hold out
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15)

# build pipeline
pipeline = set_pipeline()

# train the pipeline
train(X_train, y_train, pipeline)

# evaluate the pipeline
rmse = evaluate(X_val, y_val, pipeline)
print(f"RMSE: {rmse}")

RMSE: 6.24303595614125


### Congrats!

Now we are ready to convert this complete workflow into a packaged code 🚀